<h1>8장 시맨틱 검색과 RAG</h1>
<i>LLM의 핵심 요소인 검색을 살펴 봅니다.</i>

<a href="https://github.com/rickiepark/handson-llm"><img src="https://img.shields.io/badge/GitHub%20Repository-black?logo=github"></a>
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rickiepark/handson-llm/blob/main/chapter08.ipynb)

---

이 노트북은 <[핸즈온 LLM](https://tensorflow.blog/handson-llm/)> 책 8장의 코드를 담고 있습니다.

---

<a href="https://tensorflow.blog/handson-llm/">
<img src="https://tensorflow.blog/wp-content/uploads/2025/05/ed95b8eca688ec98a8_llm.jpg" width="350"/></a>

### [선택사항] - <img src="https://colab.google/static/images/icons/colab.png" width=100>에서 패키지 선택하기


이 노트북을 구글 코랩에서 실행한다면 다음 코드 셀을 실행하여 이 노트북에서 필요한 패키지를  설치하세요.

---

💡 **NOTE**: 이 노트북의 코드를 실행하려면 GPU를 사용하는 것이 좋습니다. 구글 코랩에서는 **런타임 > 런타임 유형 변경 > 하드웨어 가속기 > T4 GPU**를 선택하세요.

---

In [ ]:
# 깃허브에서 위젯 상태 오류를 피하기 위해 진행 표시줄을 나타내지 않도록 설정합니다.
import os
import tqdm
from transformers.utils import logging

# tqdm 비활성화
# tqdm.tqdm = lambda *args, **kwargs: iter([])
# tqdm.auto.tqdm = lambda *args, **kwargs: iter([])
# tqdm.notebook.tqdm = lambda *args, **kwargs: iter([])
os.environ["DISABLE_TQDM"] = "1"

logging.disable_progress_bar()

In [ ]:
%%capture
!pip install cohere faiss-cpu rank_bm25 langchain-community

# 사용하는 파이썬과 CUDA 버전에 맞는 llama-cpp-python 패키지를 설치하세요.
# 현재 코랩의 파이썬 버전은 3.11이며 CUDA 버전은 12.4입니다.
!pip install https://github.com/abetlen/llama-cpp-python/releases/download/v0.3.4-cu124/llama_cpp_python-0.3.4-cp311-cp311-linux_x86_64.whl

## 밀집 검색 예제

### 1. 텍스트 문서 분할하기


In [ ]:
import cohere

# 코히어의 API 키를 입력하세요.
api_key = 'hrnLgkfhcwNuYseUyoUP1D2DO6lCSxSkb2jNyC14'

# 코히어 클라이언트를 만듭니다.
co = cohere.Client(api_key)

In [ ]:
text = """
Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan.
It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine.
Set in a dystopian future where humanity is struggling to survive, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind.

Brothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007.
Caltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar.
Cinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm.
Principal photography began in late 2013 and took place in Alberta, Iceland, and Los Angeles.
Interstellar uses extensive practical and miniature effects and the company Double Negative created additional digital effects.

Interstellar premiered on October 26, 2014, in Los Angeles.
In the United States, it was first released on film stock, expanding to venues using digital projectors.
The film had a worldwide gross over $677 million (and $773 million with subsequent re-releases), making it the tenth-highest grossing film of 2014.
It received acclaim for its performances, direction, screenplay, musical score, visual effects, ambition, themes, and emotional weight.
It has also received praise from many astronomers for its scientific accuracy and portrayal of theoretical astrophysics. Since its premiere, Interstellar gained a cult following,[5] and now is regarded by many sci-fi experts as one of the best science-fiction films of all time.
Interstellar was nominated for five awards at the 87th Academy Awards, winning Best Visual Effects, and received numerous other accolades"""

# 문장을 나누어 리스트로 만듭니다.
texts = text.split('.')

# 공백과 줄바꿈 문자를 삭제합니다.
texts = [t.strip(' \n') for t in texts]

### 2. 문장을 임베딩하기


In [ ]:
import numpy as np

# 임베딩을 만듭니다.
response = co.embed(
  texts=texts,
  input_type="search_document",
).embeddings

embeds = np.array(response)
print(embeds.shape)

### 3. 검색 인덱스 구축하기


In [ ]:
import faiss

dim = embeds.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(np.float32(embeds))

### 4. 인덱스 검색하기


In [ ]:
import pandas as pd

def search(query, number_of_results=3):

  # 1. 쿼리의 임베딩을 만듭니다.
  query_embed = co.embed(texts=[query],
                input_type="search_query",).embeddings[0]

  # 2. 최근접 이웃을 추출합니다.
  distances, similar_item_ids = index.search(np.float32([query_embed]),
                                             number_of_results)

  # 3. 데이터프레임을 사용해 출력을 준비합니다.
  texts_np = np.array(texts) # 인덱싱을 쉽게 하기 위해 텍스트 리스트를 넘파이 배열로 변환합니다.
  results = pd.DataFrame(data={'텍스트': texts_np[similar_item_ids[0]],
                               '거리': distances[0]})

  # 4. 결과를 출력하고 반환합니다.
  print(f"쿼리:'{query}'\n최근접 이웃:")
  return results

In [ ]:
query = "how precise was the science"
results = search(query)
results

In [ ]:
from rank_bm25 import BM25Okapi
from sklearn.feature_extraction import _stop_words
import string

def bm25_tokenizer(text):
    tokenized_doc = []
    for token in text.lower().split():
        token = token.strip(string.punctuation)

        if len(token) > 0 and token not in _stop_words.ENGLISH_STOP_WORDS:
            tokenized_doc.append(token)
    return tokenized_doc

In [ ]:
from tqdm import tqdm

tokenized_corpus = []
for passage in tqdm(texts):
    tokenized_corpus.append(bm25_tokenizer(passage))

bm25 = BM25Okapi(tokenized_corpus)

In [ ]:
def keyword_search(query, top_k=3, num_candidates=15):
    print("입력 질문:", query)

    ##### BM25 검색 (어휘 검색) #####
    bm25_scores = bm25.get_scores(bm25_tokenizer(query))
    top_n = np.argpartition(bm25_scores, -num_candidates)[-num_candidates:]
    bm25_hits = [{'corpus_id': idx, 'score': bm25_scores[idx]} for idx in top_n]
    bm25_hits = sorted(bm25_hits, key=lambda x: x['score'], reverse=True)

    print(f"탑-3 어휘 검색 (BM25) 결과")
    for hit in bm25_hits[0:top_k]:
        print("\t{:.3f}\t{}".format(hit['score'], texts[hit['corpus_id']].replace("\n", " ")))

In [ ]:
keyword_search(query = "how precise was the science")

### 밀집 검색의 단점


In [ ]:
query = "What is the mass of the moon?"
results = search(query)
results

## 리랭킹 예제


In [ ]:
query = "how precise was the science"
results = co.rerank(query=query, documents=texts, top_n=3, return_documents=True)
results.results

In [ ]:
for idx, result in enumerate(results.results):
    print(idx, result.relevance_score , result.document.text)

In [ ]:
def keyword_and_reranking_search(query, top_k=3, num_candidates=10):
    print("입력 질문:", query)

    ##### BM25 검색 (어휘 검색) #####
    bm25_scores = bm25.get_scores(bm25_tokenizer(query))
    top_n = np.argpartition(bm25_scores, -num_candidates)[-num_candidates:]
    bm25_hits = [{'corpus_id': idx, 'score': bm25_scores[idx]} for idx in top_n]
    bm25_hits = sorted(bm25_hits, key=lambda x: x['score'], reverse=True)

    print(f"탑-3 어휘 검색 (BM25) 결과")
    for hit in bm25_hits[0:top_k]:
        print("\t{:.3f}\t{}".format(hit['score'], texts[hit['corpus_id']].replace("\n", " ")))

    # 리랭킹 추가
    docs = [texts[hit['corpus_id']] for hit in bm25_hits]

    print(f"\n리랭킹으로 얻은 탑-3 결과 ({len(bm25_hits)}개의 BM25 결과를 재조정함)")
    results = co.rerank(query=query, documents=docs, top_n=top_k, return_documents=True)
    for hit in results.results:
        print("\t{:.3f}\t{}".format(hit.relevance_score, hit.document.text.replace("\n", " ")))

In [ ]:
keyword_and_reranking_search(query = "how precise was the science")

## RAG

### 예: LLM API를 사용한 근거 기반 생성


In [ ]:
query = "income generated"

# 1- 검색
# 여기서는 임베딩 검색을 사용하지만 하이브리드 방식이 이상적입니다.
results = search(query)

# 2- 근거 기반 생성
docs_dict = [{'text': text} for text in results['텍스트']]
response = co.chat(
    message = query,
    documents=docs_dict
)

print(response.text)

In [ ]:
response

In [ ]:
response.citations

### 예: 로컬 모델을 사용한 RAG


#### 생성 모델 로드하기


In [ ]:
!wget https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-q4.gguf

In [ ]:
from langchain import LlamaCpp

# 여러분의 컴퓨터에 다운로드한 모델의 경로를 입력하세요!
llm = LlamaCpp(
    model_path="Phi-3-mini-4k-instruct-q4.gguf",
    n_gpu_layers=-1,
    max_tokens=500,
    n_ctx=4096,
    seed=42,
    verbose=False
)

#### 임베딩 모델 로드하기

In [ ]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

# 텍스트를 수치 표현으로 변환하기 위한 임베딩 모델
embedding_model = HuggingFaceEmbeddings(
    model_name='BAAI/bge-small-en-v1.5'
)

#### 벡터 데이터베이스 구축

In [ ]:
from langchain.vectorstores import FAISS

# 로컬 벡터 데이터베이스를 만듭니다.
db = FAISS.from_texts(texts, embedding_model)

#### RAG 프롬프트


In [ ]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA


# 프롬프트 템플릿을 만듭니다.
template = """<|user|>
Relevant information:
{context}

Provide a concise answer the following question using the relevant information provided above:
{question}<|end|>
<|assistant|>"""
prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
)

# RAG 파이프라인
rag = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=db.as_retriever(),
    chain_type_kwargs={
        "prompt": prompt
    },
    verbose=True
)

In [ ]:
rag.invoke('Income generated')